In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-aryr1103')

In [2]:
def dataset_value(dataset, lon, lat):
    point = ee.Geometry.Point([lon, lat])
    if isinstance(dataset, ee.imagecollection.ImageCollection):
        dataset = dataset.mean()
    value = dataset.reduceRegion(
        reducer=ee.Reducer.first(), 
        geometry=point, 
        scale=30
    ).getInfo()
    return value
startYear = '2020-07-01'
endYear = '2020-09-01'

In [3]:
elevation = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
temperature = ee.ImageCollection("MODIS/061/MOD11A1").select('LST_Day_1km')
biomass = ee.ImageCollection("WCMC/biomass_carbon_density/v1_0").select('carbon_tonnes_per_ha')
humidity = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H").select('Qair_f_inst')
precipitation = ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06").select('precipitation')
fires = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1")

In [4]:
n=479
fire_feature = ee.Feature(fires.toList(n).get(n-1))
fire_bounds = fire_feature.geometry().bounds()
buffer_distance = 1

# Create a rectangle around the fire perimeter with a buffer
rectangle = fire_bounds.buffer(buffer_distance).bounds()

In [5]:
def generate_grid(corners, point_spacing):
    coords = []
    min_lon, min_lat = corners[0]
    max_lon, max_lat = corners[1]

    lon = min_lon
    while lon < max_lon:
        lat = min_lat
        while lat < max_lat:
            coords.append([lon, lat])
            lat += point_spacing
        lon += point_spacing
    return coords


In [6]:
# Define the spacing between points (in degrees)
point_spacing = 0.01  # Example: approximately 111 meters at the equator

# Extract the lower-left and upper-right corners of the rectangle
rect_coords = rectangle.bounds().getInfo()['coordinates'][0]
lower_left = rect_coords[0]  # Lower-left corner
upper_right = rect_coords[2]  # Upper-right corner
    
grid_points = generate_grid([lower_left, upper_right], point_spacing)
print(lower_left, upper_right)
center = [0, 0]
center[0] = (lower_left[0]+upper_right[0])/2
center[1] = (lower_left[1]+upper_right[1])/2
print(center)
print(len(grid_points))

[-115.56751081403489, 42.39835008521034] [-114.88116823901456, 42.92725758477888]
[-115.22433952652472, 42.662803834994605]
3657


In [7]:
import geemap.foliumap as geemap
import folium
Map = geemap.Map()
for point in grid_points:
    color = 'blue'
    if fire_feature.geometry().contains(ee.Geometry.Point(point)).getInfo():
        color = 'red'
    folium.Circle(
        location=point[::-1],
        radius=10,
        color=color
    ).add_to(Map)
Map

KeyboardInterrupt: 

In [8]:
import os
import random
import csv

# Define a maximum number of iterations to prevent an infinite loop
max_iterations = 1000000000  # example value
csv_file_path = 'rishyorz.csv'

# Check if the file exists and is empty
is_file_empty = not os.path.exists(csv_file_path) or os.stat(csv_file_path).st_size == 0

# Open a new CSV file
with open(csv_file_path, 'a', newline='') as file:  # 'a' mode appends to the file if it exists
    writer = csv.writer(file)
    
    # Write header only if the file is empty
    if is_file_empty:
        writer.writerow(['Longitude', 'Latitude', 'Elevation', 'Temperature', 'Biomass', 'Humidity', 'Percipitation', 'Burned'])

    for _ in range(max_iterations):
        i = random.randint(0, len(grid_points)-1)
        lon = grid_points[i][0]
        lat = grid_points[i][1]
        elev = dataset_value(elevation, lon, lat)
        temp = dataset_value(temperature, lon, lat)
        bio = dataset_value(biomass, lon, lat)
        hum = dataset_value(humidity, lon, lat)
        precip = dataset_value(precipitation, lon, lat)
        burned = fire_feature.geometry().contains(ee.Geometry.Point([lon, lat])).getInfo()
        writer.writerow([lon, lat, elev['elevation'], temp['LST_Day_1km'], bio['carbon_tonnes_per_ha'], hum['Qair_f_inst'], precip['precipitation'], burned])

        # Optionally, flush data to file after each write
        file.flush()
